---



In [1]:
# Setup
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

# Always prompt for the API key if it's not set or invalid
if "GROQ_API_KEY" not in os.environ or not os.environ["GROQ_API_KEY"]:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b (Small/Fast) to demonstrate logic failures
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.0)

Enter your Groq API Key: ··········


In [2]:
question = "Ramesh has 5 cricket balls. He buys 2 more cans of cricket balls. Each can has 3 cricket balls. How many does he have now?"

# 1. Standard Prompt (Direct Answer)
prompt_standard = f"Answer this question: {question}"
print("--- STANDARD (Llama3.1-8b) ---")
print(llm.invoke(prompt_standard).content)

--- STANDARD (Llama3.1-8b) ---
To find out how many cricket balls Ramesh has now, we need to add the initial number of cricket balls he had (5) to the number of cricket balls he bought (2 * 3 = 6).

So, Ramesh initially had 5 cricket balls. He bought 2 cans of 3 cricket balls each, which is 6 cricket balls. 

Now, Ramesh has 5 + 6 = 11 cricket balls.


In [3]:
# 2. CoT Prompt (Magic Phrase)
prompt_cot = f"Answer this question. Let's think step by step. {question}"

print("--- Chain of Thought (Llama3.1-8b) ---")
print(llm.invoke(prompt_cot).content)

--- Chain of Thought (Llama3.1-8b) ---
To find out how many cricket balls Ramesh has now, we need to follow these steps:

1. Ramesh already has 5 cricket balls.
2. He buys 2 more cans of cricket balls. Each can has 3 cricket balls, so he buys 2 x 3 = 6 more cricket balls.
3. Now, we add the cricket balls he already had (5) to the new cricket balls he bought (6). So, 5 + 6 = 11.

Therefore, Ramesh now has 11 cricket balls.


---



In [4]:
# Setup
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

# Always prompt for the API key if it's not set or invalid
if "GROQ_API_KEY" not in os.environ or not os.environ["GROQ_API_KEY"]:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.7) # Creativity needed

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

problem = "How can I get my 10-year-old to study?"

# Step 1: The Branch Generator
prompt_branch = ChatPromptTemplate.from_template(
    "Problem: {problem}. Give me one unique, creative solution. Solution {id}:"
)

branches = RunnableParallel(
    sol1=prompt_branch.partial(id="1") | llm | StrOutputParser(),
    sol2=prompt_branch.partial(id="2") | llm | StrOutputParser(),
    sol3=prompt_branch.partial(id="3") | llm | StrOutputParser(),
)

# Step 2: The Judge
prompt_judge = ChatPromptTemplate.from_template(
    """
    I have three proposed solutions for: '{problem}'

    1: {sol1}
    2: {sol2}
    3: {sol3}

    Act as a Child Psychologist. Pick the most sustainable one (not bribery) and explain why.
    """
)

# Chain: Input -> Branches -> Judge -> Output
tot_chain = (
    RunnableParallel(problem=RunnableLambda(lambda x: x), branches=branches)
    | (lambda x: {**x["branches"], "problem": x["problem"]})
    | prompt_judge
    | llm
    | StrOutputParser()
)

print("--- Tree of Thoughts (ToT) Result ---")
print(tot_chain.invoke(problem))

--- Tree of Thoughts (ToT) Result ---
As a child psychologist, I would recommend Solution 2: "The Gamified Study Session" as the most sustainable approach. Here's why:

1. **Encourages intrinsic motivation**: By setting up a system where your child earns points and rewards based on their effort and progress, you're encouraging them to develop an intrinsic motivation to study. This means they'll be more likely to enjoy learning for its own sake, rather than just for external rewards.
2. **Develops self-regulation skills**: The gamified approach helps your child develop self-regulation skills, such as setting goals, tracking progress, and managing their time. These skills are essential for academic success and will benefit them throughout their lives.
3. **Fosters a growth mindset**: By framing learning as a game, you're promoting a growth mindset, where your child sees challenges as opportunities for growth and development, rather than threats to their ego.
4. **Encourages self-reflecti

In [6]:
# 1. The Generator (Divergence)
prompt_draft = ChatPromptTemplate.from_template(
    "Write a 1-sentence movie plot about: {topic}. Genre: {genre}."
)

drafts = RunnableParallel(
    draft_scifi=prompt_draft.partial(genre="Sci-Fi") | llm | StrOutputParser(),
    draft_romance=prompt_draft.partial(genre="Romance") | llm | StrOutputParser(),
    draft_horror=prompt_draft.partial(genre="Horror") | llm | StrOutputParser(),
)

# 2. The Aggregator (Convergence)
prompt_combine = ChatPromptTemplate.from_template(
    """
    I have three movie ideas for the topic '{topic}':
    1. Sci-Fi: {draft_scifi}
    2. Romance: {draft_romance}
    3. Horror: {draft_horror}

    Your task: Create a new Mega-Movie that combines the TECHNOLOGY of Sci-Fi, the PASSION of Romance, and the FEAR of Horror.
    Write one paragraph.
    """
)

# 3. The Chain
got_chain = (
    RunnableParallel(topic=RunnableLambda(lambda x: x), drafts=drafts)
    | (lambda x: {**x["drafts"], "topic": x["topic"]})
    | prompt_combine
    | llm
    | StrOutputParser()
)

print("--- Graph of Thoughts (GoT) Result ---")
print(got_chain.invoke("Time Travel"))

--- Graph of Thoughts (GoT) Result ---
In the mind-bending thriller "Echoes of Eternity," brilliant physicist Dr. Sophia Ellis discovers a way to manipulate time, but her experiment goes catastrophically wrong, causing her consciousness to become trapped in a never-ending loop of past, present, and future. As she frantically tries to escape the paradox, she finds herself reliving the same pivotal moments in history, from the moon landing to the sinking of the Titanic, but each time with a twist - she's not alone. A mysterious and enigmatic figure, the "Chrono," seems to be manipulating events from behind the scenes, and Sophia soon realizes that her only hope for survival lies in unraveling the mystery of the Chrono's true identity, a dashing and charismatic artist from a bygone era who may hold the key to her escape, but at a terrible cost: her own soul.
